In [2]:
from gurobipy import Model, GRB, quicksum
import pandas as pd
import numpy as np

def read_basic_data():
    # Excel file names
    travel_times = pd.read_excel('a2_part1.xlsx','Travel Times')
    lines = pd.read_excel('a2_part1.xlsx', 'Lines')

    lines = lines.drop(['Name','Frequency'], axis = 1)
    lines = lines.fillna(0)

    weights = 1 #Each activity has the same importance for us
    
    return travel_times, lines, weights



In [3]:
def build_model(travel_times, lines, weights):
    T = 30

    # Initialize the Gurobi model
    model = Model("NS_trains")

    line_names = ['800','3000','3100','3500','3900']
    running_activites_f = {}
    running_activites_b = {}
    dwelling_activities_f = {}
    dwelling_activities_b = {}
    transfer_activities = {}
    sync_activities = {}
    headway_activities = {}
    events_pi_f= {}
    events_pi_b= {}
    pij = {}

    for k in range(5):
        station_names = lines.iloc[k].tolist()  # Extract station names from the first row
        number = line_names[k]

        # Create Gurobi variables for each possible trip between stations
        for i in range(len(station_names) - 1):
            station1 = str(station_names[i])
            station2 = str(station_names[i+1])
            station3 = str(station_names[i-1])

            if station1 == '0' or station2 == '0':
                break
                
            # Running activities forward (ra)
            running_activites_f[f'ra-{station1}-{station2}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                        name=f'ra-{station1}-{station2}-{number}')
            
            # Running activities backwards (ra)
            running_activites_b[f'ra-{station2}-{station1}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                        name=f'ra-{station2}-{station1}-{number}')
            
            pij[f'ra-{station1}-{station2}-{number}'] = model.addVar(vtype=GRB.INTEGER,
                                                                        name=f'pra-{station1}-{station2}-{number}')
            
            # Running activities backwards (ra)
            pij[f'ra-{station2}-{station1}-{number}'] = model.addVar(vtype=GRB.INTEGER,
                                                                        name=f'pra-{station2}-{station1}-{number}')


            if i != 0 and i != len(station_names):
                # Dwelling activities forward (daf)
                dwelling_activities_f[f'daf-{station3}-{station1}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f'daf-{station3}-{station1}-{number}')
                # Dwelling activities backwards (dab)
                dwelling_activities_b[f'dab-{station2}-{station1}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f'dab-{station2}-{station1}-{number}')
                
                
                # Dwelling activities forward (daf)
                pij[f'daf-{station3}-{station1}-{number}'] = model.addVar(vtype=GRB.INTEGER,
                                                        name=f'pdaf-{station3}-{station1}-{number}')
                # Dwelling activities backwards (dab)
                pij[f'dab-{station2}-{station1}-{number}'] = model.addVar(vtype=GRB.INTEGER,
                                                        name=f'pdab-{station2}-{station1}-{number}')
    
    
            # Events
            # Arrival events forward (aef)
            events_pi_f[f'arr-{station1}-{station2}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                            name=f'arr-{station1}-{station2}-{number}')
            # Departure events forward (def)
            events_pi_f[f'dep-{station1}-{station2}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                             name=f'dep-{station1}-{station2}-{number}')
            # Arrival events forward (aeb)
            events_pi_b[f'arr-{station2}-{station1}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                            name=f'arr-{station2}-{station1}-{number}')
            
            # Departure events backwards (deb)
            events_pi_b[f'dep-{station2}-{station1}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                            name=f'dep-{station2}-{station1}-{number}')
            
            
    # # Four Transfer activities at Eindhoven
    # # Transfer activities backwards (tab)    
    # transfer_activities[f'trans-Hrl-Ut-3500'] = model.addVar(vtype=GRB.CONTINUOUS,
    #                                                     name=f'trans-Hrl-Ut-3500')
    # transfer_activities[f'trans-Hrl-Ut-800'] = model.addVar(vtype=GRB.CONTINUOUS,
    #                                                     name=f'trans-Hrl-Ut-800')
    # 
    # # Transfer activities forward (taf)    
    # transfer_activities[f'trans-Ut-Hrl-800'] = model.addVar(vtype=GRB.CONTINUOUS,
    #                                                     name=f'trans-Ut-Hrl-800')
    # transfer_activities[f'trans-Ut-Hrl-3500'] = model.addVar(vtype=GRB.CONTINUOUS,
    #                                                     name=f'trans-Ut-Hrl-3500')
    # 
    # 
    # # Transfer activities backwards (tab)    
    # pij[f'trans-Hrl-Ut-3500'] = model.addVar(vtype=GRB.CONTINUOUS,
    #                                                     name=f'ptrans-Hrl-Ut-3500')
    # pij[f'trans-Hrl-Ut-800'] = model.addVar(vtype=GRB.CONTINUOUS,
    #                                                     name=f'ptrans-Hrl-Ut-800')
    # 
    # # Transfer activities forward (taf)    
    # pij[f'trans-Ut-Hrl-800'] = model.addVar(vtype=GRB.CONTINUOUS,
    #                                                     name=f'ptrans-Ut-Hrl-800')
    # pij[f'trans-Ut-Hrl-3500'] = model.addVar(vtype=GRB.CONTINUOUS,
    #                                                     name=f'ptrans-Ut-Hrl-3500')
    # 
    # headway_pairs = [
    # ('800', '3500'),
    # ('800', '3100'),
    # ('3000', '3500'),
    # ('3000', '3100')
    # ]
    # 
    # #Headway activities arrival or departure
    # activity_types = ['a', 'd']
    # 
    # # ha (a/d)
    # for pair in headway_pairs:
    #     for activity_type in activity_types:
    #         key = f'ha{activity_type}-{pair[0]}-{pair[1]}'
    #         headway_activities[key] = model.addVar(vtype=GRB.CONTINUOUS, name=key)
    # 
    # 
    # # Sync activities forwards/backwards (saf/sab)
    # sync_pairs = [
    # ('Amr_Asd_800', 'Amr_Asd_3000'),
    # ('Asd_Ut_800', 'Asd_Ut_3000'),
    # ('Ut_Ehv_800', 'Ut_Ehv_3500'),
    # ('Shl_Ut_3100', 'Shl_Ut_3500'),
    # ('Ut_Nm_3000', 'Ut_Nm_3100'),
    # ('Ehv_Std_800', 'Ehv_Std_3900')
    # ]
    # 
    # # Define the types of headway activities
    # direction = ['f', 'b']
    # 
    # # Iterate over each pair and activity type to create variables
    # for pair in sync_pairs:
    #     for d in direction:
    #         key = f'sa{d}-{pair[0]}-{pair[1]}'
    #         sync_activities[key] = model.addVar(vtype=GRB.CONTINUOUS, name=key)


    # # Objective function: Minimize the total weighted travel time
    # model.setObjective(
    # quicksum(running_activites_f.values()) + 
    # quicksum(dwelling_activities_f.values()) + 
    # quicksum(transfer_activities.values()) + 
    # quicksum(headway_activities.values()) + 
    # quicksum(sync_activities.values()) + 
    # quicksum(running_activites_b.values())+ 
    # quicksum(dwelling_activities_b.values()), GRB.MINIMIZE)
    
        # Objective function: Minimize the total weighted travel time
    model.setObjective(
    quicksum(running_activites_f.values()) + 
    quicksum(dwelling_activities_f.values()) + 
    quicksum(running_activites_b.values())+ 
    quicksum(dwelling_activities_b.values()), GRB.MINIMIZE)
        
    
    # Constrains   
    # Running time  LB/UB
    for k, activity_var in running_activites_f.items():
        parts = k.split('-')
        station1, station2 = parts[1], parts[2]
        result = travel_times.loc[(travel_times['From'] == station1) & (travel_times['To'] == station2), 'Travel Time'].values[0]
        model.addConstr(activity_var == result , name=f'{k}_constrain')
    
    
    # Running time  LB/UB
    for k,activity_var in running_activites_b.items():
        parts = k.split('-')
        station1, station2 = parts[1], parts[2]
        result = travel_times.loc[(travel_times['From'] == station2) & (travel_times['To'] == station1), 'Travel Time'].values[0]
        model.addConstr(activity_var == result, name=f'{k}_constrain')
        
        
    # Running time constrains Pi + T  
    for k, activity_var in running_activites_f.items():
        parts = k.split('-')
        station1, station2, number = parts[1], parts[2], parts[3]
        
        # Find the corresponding events for this activity
        start_event = events_pi_f[f'dep-{station1}-{station2}-{number}']
        end_event = events_pi_f[f'arr-{station1}-{station2}-{number}']
        
        # Create the constraint for this activity
        model.addConstr(activity_var == end_event - start_event + T * pij[k], name=f"duration_{k}")
    
    
    # Running time constrains Pi + T
    for k, activity_var in running_activites_b.items():
        # Parse activity keys to get the station names and number
        parts = k.split('-')
        station1, station2, number = parts[1], parts[2], parts[3]
        
        # Find the corresponding events for this activity
        start_event = events_pi_b[f'dep-{station1}-{station2}-{number}']
        end_event = events_pi_b[f'arr-{station1}-{station2}-{number}']
        
        # Create the constraint for this activity
        model.addConstr(activity_var == end_event - start_event + T * pij[k], name=f"duration_{k}")
        
        
    # Dwelling time constrains LB/UB
    for k,value in dwelling_activities_f.items():
        model.addConstr(2 <= dwelling_activities_f[k],
                name=f'{k}_constrain_2')
        model.addConstr(dwelling_activities_f[k] <= 8,
                name=f'{k}_constrain_8')
    
    # Dwelling time constrains LB/UB
    for k,value in dwelling_activities_b.items():
        model.addConstr(2 <= dwelling_activities_b[k],
                name=f'{k}_constrain_2')
        model.addConstr(dwelling_activities_b[k] <= 8,
                name=f'{k}_constrain_8')

    # Dwelling time constrains Pi + T
    for k, d_var in dwelling_activities_f.items():
        # Parse activity keys to get the station names and number
        parts = k.split('-')
        station1, station2, number = parts[1], parts[2], parts[3]
    
        # Find the corresponding events for this activity
        start_event = events_pi_f[f'arr-{station1}-{station2}-{number}']
        end_event = events_pi_f[f'dep-{station1}-{station2}-{number}']
        
        # Create the constraint for this activity
        model.addConstr(d_var == end_event - start_event + T * pij[k], name=f"{k}")

    # Dwelling time constrains Pi + T
    for k, d_var in dwelling_activities_b.items():
        # Parse activity keys to get the station names and number
        parts = k.split('-')
        station1, station2, number = parts[1], parts[2], parts[3]
    
        # Find the corresponding events for this activity
        start_event = events_pi_b[f'arr-{station1}-{station2}-{number}']
        end_event = events_pi_b[f'dep-{station1}-{station2}-{number}']
        
        # Create the constraint for this activity
        model.addConstr(d_var == end_event - start_event + T * pij[k], name=f"{k}")
        
        
    for k,value in events_pi_f.items():
        model.addConstr(0 <= events_pi_f[k],
                name=f'{k}_constrain_0')
        model.addConstr(events_pi_f[k] <= T,
                name=f'{k}_constrain_T')
        
    for k,value in events_pi_b.items():
        model.addConstr(0 <= events_pi_b[k],
                name=f'{k}_constrain_0')
        model.addConstr(events_pi_b[k] <= T,
                name=f'{k}_constrain_T')



    model.update()
    print("Constraints:")
    for constr in model.getConstrs():
        print(f"{constr.ConstrName}: {constr.sense} {constr.RHS}")

    
    return model

travel_times, lines, weights = read_basic_data()
model = build_model(travel_times, lines, weights)


Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-20
Constraints:
ra-Amr-Asd-800_constrain: = 35.0
ra-Asd-Ut-800_constrain: = 26.0
ra-Ut-Ehv-800_constrain: = 48.0
ra-Ehv-Std-800_constrain: = 47.0
ra-Std-Mt-800_constrain: = 14.0
ra-Hdr-Amr-3000_constrain: = 37.0
ra-Amr-Asd-3000_constrain: = 35.0
ra-Asd-Ut-3000_constrain: = 26.0
ra-Ut-Nm-3000_constrain: = 52.0
ra-Shl-Ut-3100_constrain: = 30.0
ra-Ut-Nm-3100_constrain: = 52.0
ra-Shl-Ut-3500_constrain: = 30.0
ra-Ut-Ehv-3500_constrain: = 48.0
ra-Ehv-Vl-3500_constrain: = 39.0
ra-Ehv-Std-3900_constrain: = 47.0
ra-Std-Hrl-3900_constrain: = 15.0
ra-Asd-Amr-800_constrain: = 35.0
ra-Ut-Asd-800_constrain: = 26.0
ra-Ehv-Ut-800_constrain: = 48.0
ra-Std-Ehv-800_constrain: = 47.0
ra-Mt-Std-800_constrain: = 14.0
ra-Amr-Hdr-3000_constrain: = 37.0
ra-Asd-Amr-3000_constrain: = 35.0
ra-Ut-Asd-3000_constrain: = 26.0
ra-Nm-Ut-3000_constrain: = 52.0
ra-Ut-Shl-3100_constrain: = 30.0
ra-Nm-Ut-3100_constrain: = 52.0
ra-Ut-Shl

In [6]:
def solve_model(model):

    # Optimize the model
    model.optimize()

    #Store solutions in a dictionary
    solution_dict = {}

    #Collecting solutions for decision variables
    for var in model.getVars():
        solution_dict[var.varName] = var.x

    #Store Obj.function cost
    cost = model.objVal

    #Close the Gurobi model
    model.close()

    return solution_dict, cost

In [7]:

solution_dict, cost = solve_model(model)


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 258 rows, 172 columns and 420 nonzeros
Model fingerprint: 0x1081b1a6
Variable types: 118 continuous, 54 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 5e+01]
Presolve removed 236 rows and 150 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


AttributeError: Unable to retrieve attribute 'x'